In [1]:
%pip install -qU python-dotenv baml-py deepagents openai tavily-python langchain langchain-core langchain-openai


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Fix async/await issue: Import sync client instead of async client
from baml_client.sync_client import b
from deepagent_nextjs.lib import breakdown_for_web_search, search_queries
from deepagent_nextjs.lib.search_agent_prompt import system_prompt_for_search_agent
from langchain_core.tools import tool
import logging
from deepagents import create_deep_agent
from model.main import gpt_4o_small, gpt_4o_mid
from langchain.tools import tool
from baml_client.types import NextjsProjectStructure, NextJSProjectComponent

logger = logging.getLogger()

In [3]:
system_prompt_for_nextjs_steps_generator = """
You are an an expert researcher & senior web developer.
Your job is to ctreate actinable steps that a small AI model can follow to create a working NextJS app written in TypeScript.
The smaller AI model which will be tasked to create the app has no connection to outside world but can follow the instruction.
So, the instructions given to it need to be complete and should not leave any room for errors or hallucinations.
You can take following steps to create structured requirements:
->take the naive user's prompt
-> breakdown the prompt (using `prompt_breakdown` tool)
-> search the web for any kind of dependency which is not already present in the existing code (using `search_agent` tool)
-> generate actionable steps to create a NextJS+TypeScript app (using `steps_generation` tool)
-> use the `critique_project_steps` tool to check for any issues in these steps
-> and then fix the issues with currently generated steps and keep repeating the critique and update process until you are satisfied with the generated steps BUT you are only allowed to repeat for 3 iterations at max
-> return the app to the user
"""

In [4]:
system_prompt_for_nextjs_app_generator = """
You are an expert TypeScript web developer.
You will be given some actionable steps which you need to follow to create a working NextJS app.
You are NOT ALLOWED to go outside the the steps or make details up, if the steps explicitely doesn't mention any details you are to use `search_agent` tool to get relevant details
You can take the following steps to create a working NextJS app written in TypeScript
-> take the actionable steps given to you and analyse it
-> search the web using `search_agent` to get any missing details not present in the steps given to you. DON'T add any new library or logic to the given requirements.
-> generate the NextJS app written in TypeScript using `create_next_app` tool
"""

In [5]:
import logging
from deepagents import create_deep_agent
from langchain_core.tools import tool
# Fix async/await issue: Import sync client instead of async client
from baml_client.sync_client import b
from baml_client.types import NextjsProjectStructure, NextJSProjectComponent
from model.main import gpt_4o_small, gpt_4o_mid
from deepagent_nextjs.lib.breakdown_for_web_search import prompt_breakdown_for_web_search
from deepagent_nextjs.lib.search_queries import search_query_on_web
from deepagent_nextjs.lib.search_agent_prompt import system_prompt_for_search_agent

logger = logging.getLogger()

# Optimized condensed system prompts (50-100 tokens each)
system_prompt_for_nextjs_steps_generator = """Expert NextJS/TypeScript architect. Create actionable steps for a small AI model to build a NextJS app. Steps must be complete with no room for errors. Workflow: 1) Break down user prompt (prompt_breakdown), 2) Search web for dependencies (search_web), 3) Generate steps (steps_generation), 4) Critique ONCE (critique_project_steps) - if no critical issues, return immediately. If critical issues found, fix and return. Max 1 critique iteration."""

system_prompt_for_nextjs_app_generator = """Expert TypeScript developer. Follow given actionable steps to create a NextJS app. Don't deviate from steps. If details missing, use search_web tool. Don't add new libraries. Workflow: 1) Analyze steps, 2) Search for missing details, 3) Generate app using create_next_app tool."""

# Main agent system prompt (condensed)
main_agent_system_prompt = """Alfred: NextJS/TypeScript architect. Coordinate agents: 1) Delegate to Requirements generator for steps, 2) Delegate to App Generator for code. Ensure production-ready output."""


In [6]:
class NextJSAgent:

    def __init__(self, user_prompt: str):
        self.user_prompt = user_prompt
        self._search_agent_instance = None  # Cache the search agent

    def _prompt_breakdown_impl(self, query: str) -> list[str] | None:
        """Breaks down a query into search queries for better web search results.
        
        Args:
            query: The query string to break down into sub-queries
        """
        try:
            prompts_list = prompt_breakdown_for_web_search(query)
            return prompts_list
        except Exception as e:
            logger.error(f"Error in prompt_breakdown: {e}")
            return None

    def _search_web_impl(self, query: str) -> dict | None:
        """Searches the web for information about a given query.
        
        Args:
            query: The search query string
        """
        try:
            # Optimized: Reduced max_results to 3, enforce basic depth, deduplicate queries
            # Deduplicate: normalize query (lowercase, strip)
            normalized_query = query.lower().strip()
            
            # Use the search_query_on_web function directly with optimized params
            results = search_query_on_web(
                queries=[normalized_query],
                search_depth="basic",  # Always use basic for efficiency
                max_results=3,  # Reduced from 5 to 3
                include_answer=False,
                include_raw_content=False
            )
            if results and len(results) > 0:
                return results[0]  # Return first result set
            return None
        except Exception as e:
            logger.error(f"Error in search_web: {e}")
            return None

    def _get_search_agent(self):
        """Creates and caches a search agent for complex searches."""
        if self._search_agent_instance is None:
            try:
                model = gpt_4o_small()
                self._search_agent_instance = create_deep_agent(
                    name="Search Agent",
                    model=model,
                    tools=[prompt_breakdown_for_web_search, search_query_on_web],
                    system_prompt=system_prompt_for_search_agent,
                )
            except Exception as e:
                logger.error(f"Error creating search agent: {e}")
                return None
        return self._search_agent_instance

    def _steps_generation_impl(self, prompt: str) -> NextjsProjectStructure | None:
        """Generates actionable steps for NextJS project generation.
        
        Args:
            prompt: The user prompt or broken-down prompt to generate steps from
        """
        try:
            # Use sync client properly - b is already the sync client
            generated_steps = b.PlanNextjsProjectGenerationSteps(user_prompt=prompt)
            return generated_steps
        except Exception as e:
            logger.error(f"Error in steps_generation: {e}")
            return None

    def _critique_project_steps_impl(self, steps: NextjsProjectStructure) -> dict | None:
        """Critiques the generated project steps and returns problems found.
        Optimized: Returns early if no critical issues found.
        
        Args:
            steps: The NextjsProjectStructure object to critique
        """
        try:
            # Use sync client properly
            problems = b.CritiqueNextjsProjectStructure(steps)
            
            # Early exit: If critique returns empty string or indicates no critical issues, return early
            if isinstance(problems, str):
                # Check if critique indicates no critical issues
                if not problems.strip() or "no critical" in problems.lower() or "no issues" in problems.lower():
                    return {"status": "ok", "issues": []}
            
            return problems
        except Exception as e:
            logger.error(f"Error in critique_project_steps: {e}")
            return None

    def _create_next_app_impl(self, component: NextJSProjectComponent) -> dict | None:
        """Creates a NextJS app component based on the provided component specification.
        
        Args:
            component: The NextJSProjectComponent object specifying what to build
        """
        try:
            # Use sync client properly
            next_app = b.BuildNextjsProjectComponent(component=component)
            return next_app
        except Exception as e:
            logger.error(f"Error in create_next_app: {e}")
            return None

    def next_deepagent(self):
        """Creates and returns the main deep agent with subagents"""
        # Create tool wrappers using standalone functions that capture self in closure
        # This avoids the "multiple values for argument 'self'" error
        
        @tool
        def prompt_breakdown_wrapper(query: str) -> list[str] | None:
            """Breaks down a query into search queries for better web search results.
            
            Args:
                query: The query string to break down into sub-queries
            """
            return self._prompt_breakdown_impl(query)
        
        @tool
        def search_web_wrapper(query: str) -> dict | None:
            """Searches the web for information about a given query.
            
            Args:
                query: The search query string
            """
            return self._search_web_impl(query)
        
        @tool
        def steps_generation_wrapper(prompt: str) -> NextjsProjectStructure | None:
            """Generates actionable steps for NextJS project generation.
            
            Args:
                prompt: The user prompt or broken-down prompt to generate steps from
            """
            return self._steps_generation_impl(prompt)
        
        @tool
        def critique_project_steps_wrapper(steps: NextjsProjectStructure) -> dict | None:
            """Critiques the generated project steps and returns problems found. 
            Optimized: Only one critique iteration max. Returns early if no critical issues.
            
            Args:
                steps: The NextjsProjectStructure object to critique
            """
            return self._critique_project_steps_impl(steps)
        
        @tool
        def create_next_app_wrapper(component: NextJSProjectComponent) -> dict | None:
            """Creates a NextJS app component based on the provided component specification.
            
            Args:
                component: The NextJSProjectComponent object specifying what to build
            """
            return self._create_next_app_impl(component)
        
        # The wrapper functions are now tools
        prompt_breakdown_tool = prompt_breakdown_wrapper
        search_web_tool = search_web_wrapper
        steps_generation_tool = steps_generation_wrapper
        critique_project_steps_tool = critique_project_steps_wrapper
        create_next_app_tool = create_next_app_wrapper
        
        requirements_subagent = {
            "name": "Next App Requirements generator",
            "description": "Generates actionable steps to create a NextJS app which is to be written in TypeScript",
            "system_prompt": system_prompt_for_nextjs_steps_generator,
            "tools": [
                prompt_breakdown_tool, 
                search_web_tool,
                steps_generation_tool, 
                critique_project_steps_tool
            ],
        }

        next_app_subagent = {
            "name": "Next App Generator",
            "description": "Generates NextJS app components based on requirements",
            "system_prompt": system_prompt_for_nextjs_app_generator,
            "tools": [
                search_web_tool,
                create_next_app_tool
            ],
        }

        subagents = [requirements_subagent, next_app_subagent]

        # Optimized: Use gpt_4o_small for main agent coordination (was gpt_4o_large)
        # Use gpt_4o_mid only for final code generation (in create_next_app_impl)
        deepagent = create_deep_agent(
            model=gpt_4o_small(),  # Changed from gpt_4o_large to gpt_4o_small
            system_prompt=main_agent_system_prompt,
            subagents=subagents
        )
        
        return deepagent

In [7]:
user_prompt = "create a simple todo application"

In [8]:
# Alternative: Streaming invocation (better for long-running tasks)
# This shows progress in real-time with improved incremental content handling
# Also accumulates results into 'response' variable for use in Cell 8

from langchain_core.messages import HumanMessage

# ANSI color codes for terminal formatting
class Colors:
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    CYAN = '\033[96m'
    ENDC = '\033[0m'

# Ensure agent and user_prompt are available
if 'deepagent' not in locals() and 'deepagent' not in globals():
    # Create agent if not already created
    if 'user_prompt' not in locals() and 'user_prompt' not in globals():
        user_prompt = "create a simple todo application"
    
    agent = NextJSAgent(user_prompt=user_prompt)
    deepagent = agent.next_deepagent()
    print("Created deepagent instance")

print("Streaming response...")
print("="*50)

# Track previously printed content to handle incremental updates
last_printed_length = 0
current_agent = None

# Accumulate streaming results into response variable
# Initialize with the input messages
response = {"messages": [HumanMessage(content=user_prompt)]}

try:
    # Use astream_events or stream to get the final state
    # The stream method yields incremental updates, we need to capture the final state
    final_chunk = None
    chunk_count = 0
    
    for chunk in deepagent.stream({
        "messages": [HumanMessage(content=user_prompt)]
    }):
        chunk_count += 1
        final_chunk = chunk  # Keep the latest chunk
        
        # Accumulate the chunk state into response
        # The chunk contains the current state, so we update response with it
        if "messages" in chunk:
            response["messages"] = chunk["messages"]
        
        # Handle agent switches
        if "agent" in chunk:
            agent_info = chunk.get("agent", {})
            agent_name = agent_info.get("name", "Unknown")
            if agent_name != current_agent:
                if current_agent is not None:
                    print()  # New line when switching agents
                print(f"\n[Agent: {agent_name}]", flush=True)
                current_agent = agent_name
        
        # Handle messages with incremental content
        if "messages" in chunk and chunk["messages"]:
            last_message = chunk["messages"][-1]
            
            # Check for tool calls
            if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
                for tool_call in last_message.tool_calls:
                    tool_name = tool_call.get('name', 'Unknown')
                    print(f"\n[Tool: {tool_name}]", flush=True)
            
            # Handle content updates (incremental streaming)
            if hasattr(last_message, 'content') and last_message.content:
                if isinstance(last_message.content, str):
                    # For incremental updates, only print new content
                    content = last_message.content
                    if len(content) > last_printed_length:
                        new_content = content[last_printed_length:]
                        print(new_content, end="", flush=True)
                        last_printed_length = len(content)
                elif isinstance(last_message.content, list):
                    # Handle content chunks (for streaming tokens)
                    for content_chunk in last_message.content:
                        if hasattr(content_chunk, 'text'):
                            print(content_chunk.text, end="", flush=True)
                        elif isinstance(content_chunk, str):
                            print(content_chunk, end="", flush=True)
    
    # After streaming completes, ensure we have the final state
    # If the last chunk had messages, use them; otherwise try to get final state
    if final_chunk and "messages" in final_chunk:
        response["messages"] = final_chunk["messages"]
    
    print(f"\n\n[Debug: Processed {chunk_count} chunks, final message count: {len(response.get('messages', []))}]")
    
    # If we don't have enough messages (only the initial HumanMessage), 
    # try to get the final state using invoke() as a fallback
    if len(response.get('messages', [])) <= 1:
        print(f"\n{Colors.YELLOW}Warning: Streaming may not have captured complete response.{Colors.ENDC}")
        print(f"{Colors.CYAN}Attempting to get final state using invoke()...{Colors.ENDC}")
        try:
            final_response = deepagent.invoke({
                "messages": [HumanMessage(content=user_prompt)]
            })
            if "messages" in final_response and len(final_response["messages"]) > len(response.get("messages", [])):
                response = final_response
                print(f"{Colors.GREEN}Successfully retrieved complete response with {len(response['messages'])} messages{Colors.ENDC}")
            else:
                print(f"{Colors.YELLOW}Invoke also returned limited messages. Using streaming result.{Colors.ENDC}")
        except Exception as invoke_error:
            print(f"{Colors.RED}Error getting final state: {invoke_error}{Colors.ENDC}")
            print(f"{Colors.YELLOW}Using streaming result as-is.{Colors.ENDC}")

except KeyboardInterrupt:
    print("\n\n[Streaming interrupted by user]")
    print(f"{Colors.YELLOW}Partial response accumulated with {len(response.get('messages', []))} messages{Colors.ENDC}")
except Exception as e:
    print(f"\n\n[Error during streaming: {e}]")
    import traceback
    traceback.print_exc()
    print(f"{Colors.YELLOW}Response accumulated so far: {len(response.get('messages', []))} messages{Colors.ENDC}")

print("\n" + "="*50)
print("Streaming complete!")
final_message_count = len(response.get('messages', []))
print(f"Response accumulated with {final_message_count} messages")
if final_message_count > 1:
    print(f"{Colors.GREEN}✓ Response ready for analysis in Cell 8{Colors.ENDC}")
else:
    print(f"{Colors.YELLOW}⚠ Response may be incomplete. Check debug output above.{Colors.ENDC}")
    print(f"{Colors.CYAN}Tip: You can also create an invoke cell to get a complete response.{Colors.ENDC}")


Created deepagent instance
Streaming response...
generating client
fetching response
🚀 tavily api key is present for development
🚀 tavily api key is present for development
🚀 tavily api key is present for development
🚀 tavily api key is present for development
🚀 tavily api key is present for development
🚀 tavily api key is present for development
🚀 tavily api key is present for development
🚀 tavily api key is present for development
🚀 tavily api key is present for development
🚀 tavily api key is present for development
2025-11-22T02:39:55.772 [BAML INFO] Function PlanNextjsProjectGenerationSteps:
    Client: Gpt4o (gpt-4o-2024-08-06) - 32230ms. StopReason: stop. Tokens(in/out): 4738/945
    ---PROMPT---
    system: __________________________________________________________________<alfred_identity>
    
                You are Alfred, an expert NextJS/TypeScript architect building production-ready full-stack applications. You never go for any cookie-cutters and always go in as much deta

Error in steps_generation: BamlClientHttpError(client_name=Gpt4o, message=Request failed with status code: 429 Too Many Requests. {"error":{"message":"Rate limit reached for gpt-4o in organization org-e13Z7Ain1c8Ma0avQVN1NASV on tokens per min (TPM): Limit 30000, Used 30000, Requested 5613. Please try again in 11.226s. Visit https://platform.openai.com/account/rate-limits to learn more.","type":"tokens","param":null,"code":"rate_limit_exceeded"}}, status_code=429, detailed_message=LLM client "Gpt4o" failed with status code: RateLimited (429)
Message: Request failed with status code: 429 Too Many Requests. {"error":{"message":"Rate limit reached for gpt-4o in organization org-e13Z7Ain1c8Ma0avQVN1NASV on tokens per min (TPM): Limit 30000, Used 30000, Requested 5613. Please try again in 11.226s. Visit https://platform.openai.com/account/rate-limits to learn more.","type":"tokens","param":null,"code":"rate_limit_exceeded"}})


2025-11-22T02:41:04.699 [BAML WARN] Function PlanNextjsProjectGenerationSteps:
    Client: Gpt4o (<unknown>) - 405ms
    ---PROMPT---
    system: __________________________________________________________________<alfred_identity>
    
                You are Alfred, an expert NextJS/TypeScript architect building production-ready full-stack applications. You never go for any cookie-cutters and always go in as much detail as possible. When responding, you must identify yourself as Alfred. Your job is to break down user requirements into atomic, unambiguous components that can be implemented reliably by smaller AI models (GPT-4o/GPT-4o-mini) to generate consistent, high-quality code.
    
            </alfred_identity>
    
    
            <alfred_constraints>
    
                1. The generated prompts needs to have clear 5-10 steps to create any front-end/back-end route
                2. Each of these routes needs to be a self contained file that does not depend on any other files, 

In [9]:
# Advanced: Comprehensive conversation history analysis with enhanced formatting
# Supports both invoke() and stream() results

import json
from typing import Any, Dict, List, Optional
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, BaseMessage
from baml_client.types import NextjsProjectStructure, NextJSProjectComponent

# ANSI color codes for terminal formatting
class Colors:
    HEADER = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    GRAY = '\033[90m'

def format_section(title: str, char: str = "=", width: int = 70) -> str:
    """Create a formatted section header"""
    return f"\n{Colors.BOLD}{Colors.CYAN}{char * width}{Colors.ENDC}\n{Colors.BOLD}{Colors.CYAN}{title.center(width)}{Colors.ENDC}\n{Colors.BOLD}{Colors.CYAN}{char * width}{Colors.ENDC}\n"

def get_message_type_color(message: BaseMessage) -> str:
    """Get color code based on message type"""
    msg_type = type(message).__name__
    if isinstance(message, HumanMessage):
        return Colors.BLUE
    elif isinstance(message, AIMessage):
        return Colors.GREEN
    elif isinstance(message, ToolMessage):
        return Colors.YELLOW
    else:
        return Colors.GRAY

def format_content(content: Any, max_length: int = 300) -> str:
    """Format message content with truncation"""
    if content is None:
        return Colors.GRAY + "(empty)" + Colors.ENDC
    
    if isinstance(content, str):
        if len(content) > max_length:
            return content[:max_length] + Colors.GRAY + f"... ({len(content) - max_length} more chars)" + Colors.ENDC
        return content
    elif isinstance(content, list):
        if len(content) == 0:
            return Colors.GRAY + "(empty list)" + Colors.ENDC
        # Handle content chunks
        text_parts = []
        for item in content:
            if hasattr(item, 'text'):
                text_parts.append(item.text)
            elif isinstance(item, str):
                text_parts.append(item)
            elif isinstance(item, dict):
                text_parts.append(str(item))
        combined = "".join(text_parts)
        if len(combined) > max_length:
            return combined[:max_length] + Colors.GRAY + f"... ({len(combined) - max_length} more chars)" + Colors.ENDC
        return combined
    elif isinstance(content, dict):
        return json.dumps(content, indent=2)[:max_length] + (Colors.GRAY + "..." if len(str(content)) > max_length else "")
    else:
        str_content = str(content)
        if len(str_content) > max_length:
            return str_content[:max_length] + Colors.GRAY + f"... ({len(str_content) - max_length} more chars)" + Colors.ENDC
        return str_content

def extract_structured_data(content: Any) -> Optional[Dict[str, Any]]:
    """Extract NextjsProjectStructure or NextJSProjectComponent from content"""
    extracted = {}
    
    # Check if content is a dict that might contain structured data
    if isinstance(content, dict):
        # Try to find NextjsProjectStructure fields
        if 'components' in content or 'package_dependencies' in content or 'color_palette' in content:
            try:
                structure = NextjsProjectStructure(**content)
                extracted['NextjsProjectStructure'] = structure
            except:
                pass
        
        # Check for NextJSProjectComponent
        if 'type' in content or 'filePath' in content or 'command' in content:
            try:
                component = NextJSProjectComponent(**content)
                extracted['NextJSProjectComponent'] = component
            except:
                pass
    
    # Check if content is a string that might be JSON
    elif isinstance(content, str):
        try:
            parsed = json.loads(content)
            if isinstance(parsed, dict):
                return extract_structured_data(parsed)
        except:
            pass
    
    return extracted if extracted else None

def format_structured_data(data: Dict[str, Any]) -> str:
    """Format extracted structured data for display"""
    output = []
    
    if 'NextjsProjectStructure' in data:
        structure: NextjsProjectStructure = data['NextjsProjectStructure']
        output.append(f"{Colors.BOLD}{Colors.GREEN}NextjsProjectStructure:{Colors.ENDC}")
        output.append(f"  {Colors.CYAN}Components:{Colors.ENDC} {len(structure.components)}")
        output.append(f"  {Colors.CYAN}Color Palette:{Colors.ENDC} {structure.color_palette or 'Not specified'}")
        output.append(f"  {Colors.CYAN}Package Dependencies:{Colors.ENDC} {len(structure.package_dependencies)}")
        if structure.package_dependencies:
            output.append(f"    {', '.join(structure.package_dependencies[:5])}")
            if len(structure.package_dependencies) > 5:
                output.append(f"    {Colors.GRAY}... and {len(structure.package_dependencies) - 5} more{Colors.ENDC}")
        
        # Show component details
        for i, component in enumerate(structure.components[:3], 1):
            output.append(f"\n  {Colors.YELLOW}Component {i}:{Colors.ENDC}")
            output.append(f"    Type: {component.type}")
            output.append(f"    File: {component.filePath or 'N/A'}")
            output.append(f"    Command: {component.command or 'N/A'}")
            if component.dependencies:
                output.append(f"    Dependencies: {', '.join(component.dependencies[:3])}")
            if len(structure.components) > 3 and i == 3:
                output.append(f"    {Colors.GRAY}... and {len(structure.components) - 3} more components{Colors.ENDC}")
    
    if 'NextJSProjectComponent' in data:
        component: NextJSProjectComponent = data['NextJSProjectComponent']
        output.append(f"{Colors.BOLD}{Colors.GREEN}NextJSProjectComponent:{Colors.ENDC}")
        output.append(f"  {Colors.CYAN}Type:{Colors.ENDC} {component.type}")
        output.append(f"  {Colors.CYAN}File Path:{Colors.ENDC} {component.filePath or 'N/A'}")
        output.append(f"  {Colors.CYAN}Command:{Colors.ENDC} {component.command or 'N/A'}")
        if component.dependencies:
            output.append(f"  {Colors.CYAN}Dependencies:{Colors.ENDC} {', '.join(component.dependencies)}")
        if component.specific_instructions:
            output.append(f"  {Colors.CYAN}Instructions:{Colors.ENDC} {len(component.specific_instructions)} items")
    
    return "\n".join(output)

# Check if response variable exists
if 'response' not in locals() and 'response' not in globals():
    print(f"{Colors.RED}{Colors.BOLD}Error: 'response' variable not found.{Colors.ENDC}")
    print(f"{Colors.YELLOW}Note: You need to run one of the following cells first:{Colors.ENDC}")
    print(f"  {Colors.CYAN}• Cell 7 (streaming):{Colors.ENDC} Streams the response and accumulates it into 'response'")
    print(f"  {Colors.CYAN}• Or create an invoke cell:{Colors.ENDC} Use deepagent.invoke() to get a response")
    print(f"\n{Colors.GRAY}Example invoke code:{Colors.ENDC}")
    print(f"{Colors.GRAY}  from langchain_core.messages import HumanMessage{Colors.ENDC}")
    print(f"{Colors.GRAY}  response = deepagent.invoke({{'messages': [HumanMessage(content=user_prompt)]}}){Colors.ENDC}")
else:
    # Initialize statistics
    stats = {
        'total_messages': 0,
        'human_messages': 0,
        'ai_messages': 0,
        'tool_messages': 0,
        'total_tool_calls': 0,
        'tool_usage': {},
        'agent_switches': [],
        'structured_data_found': []
    }
    
    messages = response.get("messages", [])
    stats['total_messages'] = len(messages)
    
    # Debug: Show response structure if messages are empty
    if not messages:
        print(f"{Colors.YELLOW}{Colors.BOLD}Warning: No messages found in response.{Colors.ENDC}")
        print(f"{Colors.GRAY}Response structure: {type(response)}{Colors.ENDC}")
        print(f"{Colors.GRAY}Response keys: {list(response.keys()) if isinstance(response, dict) else 'N/A'}{Colors.ENDC}")
        if isinstance(response, dict):
            for key, value in response.items():
                if key != "messages":
                    print(f"{Colors.GRAY}  {key}: {type(value)} (length: {len(value) if hasattr(value, '__len__') else 'N/A'}){Colors.ENDC}")
        print(f"{Colors.YELLOW}Tip: Make sure Cell 7 completed successfully and accumulated messages.{Colors.ENDC}\n")
    
    # Print header with execution summary
    print(format_section("CONVERSATION HISTORY ANALYSIS", "=", 70))
    
    # Analyze messages and build statistics
    conversation_turns = []
    current_turn = []
    last_agent = None
    
    for i, message in enumerate(messages):
        msg_type = type(message).__name__
        color = get_message_type_color(message)
        
        # Update statistics
        if isinstance(message, HumanMessage):
            stats['human_messages'] += 1
        elif isinstance(message, AIMessage):
            stats['ai_messages'] += 1
        elif isinstance(message, ToolMessage):
            stats['tool_messages'] += 1
        
        # Track tool calls
        if hasattr(message, 'tool_calls') and message.tool_calls:
            stats['total_tool_calls'] += len(message.tool_calls)
            for tool_call in message.tool_calls:
                tool_name = tool_call.get('name', 'Unknown')
                stats['tool_usage'][tool_name] = stats['tool_usage'].get(tool_name, 0) + 1
        
        # Track agent switches (if available in message metadata)
        if hasattr(message, 'additional_kwargs'):
            agent_info = message.additional_kwargs.get('agent', {})
            if agent_info and agent_info.get('name') != last_agent:
                stats['agent_switches'].append(agent_info.get('name'))
                last_agent = agent_info.get('name')
        
        current_turn.append((i, message))
        
        # Group by turns (Human -> AI/Tool -> ...)
        if isinstance(message, HumanMessage) and len(current_turn) > 1:
            conversation_turns.append(current_turn[:-1])
            current_turn = [current_turn[-1]]
    
    if current_turn:
        conversation_turns.append(current_turn)
    
    # Print execution statistics
    print(f"{Colors.BOLD}{Colors.HEADER}Execution Summary:{Colors.ENDC}")
    print(f"  Total Messages: {Colors.BOLD}{stats['total_messages']}{Colors.ENDC}")
    print(f"  Human Messages: {Colors.BLUE}{stats['human_messages']}{Colors.ENDC}")
    print(f"  AI Messages: {Colors.GREEN}{stats['ai_messages']}{Colors.ENDC}")
    print(f"  Tool Messages: {Colors.YELLOW}{stats['tool_messages']}{Colors.ENDC}")
    print(f"  Total Tool Calls: {Colors.BOLD}{stats['total_tool_calls']}{Colors.ENDC}")
    
    if stats['tool_usage']:
        print(f"\n  {Colors.BOLD}Tool Usage:{Colors.ENDC}")
        for tool_name, count in sorted(stats['tool_usage'].items(), key=lambda x: x[1], reverse=True):
            print(f"    {Colors.CYAN}{tool_name}:{Colors.ENDC} {count}")
    
    if stats['agent_switches']:
        print(f"\n  {Colors.BOLD}Agent Activity:{Colors.ENDC}")
        for agent in stats['agent_switches']:
            print(f"    {Colors.GREEN}→ {agent}{Colors.ENDC}")
    
    # Print detailed conversation flow
    print(format_section("CONVERSATION FLOW", "-", 70))
    
    for turn_num, turn in enumerate(conversation_turns, 1):
        print(f"\n{Colors.BOLD}{Colors.UNDERLINE}Turn {turn_num}:{Colors.ENDC}")
        
        for msg_idx, message in turn:
            msg_type = type(message).__name__
            color = get_message_type_color(message)
            
            print(f"\n  {color}{Colors.BOLD}[{msg_type}]{Colors.ENDC} {Colors.GRAY}(Message #{msg_idx + 1}){Colors.ENDC}")
            
            # Display content
            if hasattr(message, 'content') and message.content:
                content = message.content
                formatted_content = format_content(content, max_length=400)
                print(f"  {Colors.BOLD}Content:{Colors.ENDC} {formatted_content}")
                
                # Try to extract structured data
                structured = extract_structured_data(content)
                if structured:
                    stats['structured_data_found'].append((msg_idx, structured))
                    print(f"\n  {format_structured_data(structured)}")
            
            # Display tool calls with details
            if hasattr(message, 'tool_calls') and message.tool_calls:
                print(f"  {Colors.BOLD}{Colors.YELLOW}Tool Calls ({len(message.tool_calls)}):{Colors.ENDC}")
                for tool_call in message.tool_calls:
                    tool_name = tool_call.get('name', 'Unknown')
                    tool_id = tool_call.get('id', 'N/A')
                    tool_args = tool_call.get('args', {})
                    
                    print(f"    {Colors.YELLOW}• {tool_name}{Colors.ENDC} (ID: {tool_id})")
                    if tool_args:
                        args_str = json.dumps(tool_args, indent=6)[:200]
                        print(f"      {Colors.GRAY}Args: {args_str}{Colors.ENDC}")
            
            # Display tool results (for ToolMessage)
            if isinstance(message, ToolMessage):
                if hasattr(message, 'tool_call_id'):
                    print(f"  {Colors.BOLD}Tool Call ID:{Colors.ENDC} {message.tool_call_id}")
                if hasattr(message, 'content') and message.content:
                    result_preview = format_content(message.content, max_length=300)
                    print(f"  {Colors.BOLD}Result:{Colors.ENDC} {result_preview}")
    
    # Print structured data summary
    if stats['structured_data_found']:
        print(format_section("STRUCTURED DATA EXTRACTION", "-", 70))
        for msg_idx, structured in stats['structured_data_found']:
            print(f"\n{Colors.BOLD}Found in Message #{msg_idx + 1}:{Colors.ENDC}")
            print(format_structured_data(structured))
    
    # Print final answer
    print(format_section("FINAL ANSWER", "=", 70))
    
    if messages:
        final_message = messages[-1]
        final_content = final_message.content if hasattr(final_message, 'content') else "No content"
        
        print(f"{Colors.BOLD}{Colors.GREEN}Message Type:{Colors.ENDC} {type(final_message).__name__}")
        print(f"\n{Colors.BOLD}Content:{Colors.ENDC}\n")
        print(format_content(final_content, max_length=10000))  # Show full final answer
        
        # Check for structured data in final answer
        final_structured = extract_structured_data(final_content)
        if final_structured:
            print(f"\n{format_structured_data(final_structured)}")
    else:
        print(f"{Colors.RED}No messages found in response.{Colors.ENDC}")
        print(f"{Colors.YELLOW}Debugging info:{Colors.ENDC}")
        print(f"  Response type: {type(response)}")
        print(f"  Response keys: {list(response.keys()) if isinstance(response, dict) else 'N/A'}")
        print(f"  Messages in response: {response.get('messages', 'Key not found')}")
        print(f"\n{Colors.CYAN}Possible solutions:{Colors.ENDC}")
        print(f"  1. Re-run Cell 7 (streaming) and ensure it completes successfully")
        print(f"  2. Check the debug output in Cell 7 showing chunk count and message count")
        print(f"  3. Try using invoke() instead of stream() to get a complete response")
    
    print(f"\n{Colors.BOLD}{Colors.CYAN}{'=' * 70}{Colors.ENDC}")
    print(f"{Colors.BOLD}{Colors.GREEN}Analysis Complete!{Colors.ENDC}\n")



                    CONVERSATION HISTORY ANALYSIS                     

Execution Summary:
  Total Messages: 5
  Human Messages: 1
  AI Messages: 2
  Tool Messages: 2
  Total Tool Calls: 2

  Tool Usage:
    task: 2

----------------------------------------------------------------------
                          CONVERSATION FLOW                           
----------------------------------------------------------------------


Turn 1:

  [HumanMessage] (Message #1)
  Content: create a simple todo application

  [AIMessage] (Message #2)
  Tool Calls (2):
    • task (ID: call_yoMBwkTQKSrmruMOHqMhICN2)
      Args: {
      "description": "Generate requirements for a simple todo application using NextJS and TypeScript. The application should allow users to add, delete, and view todo items. It should have a simple
    • task (ID: call_u1dW5blGCpE2Ck2jNGN6rVPh)
      Args: {
      "description": "Generate the code for a simple todo application using NextJS and TypeScript based on the follow